In [ ]:
import torch

__all__ = ['get_softmax_uncertainty_distance']

def get_softmax_uncertainty_distance(pred,mask = None):
  """
  pred: BS,C,H,W before softmax is applied! 
  
  distance between sec and first element
  # 1 if fully uncertain -> 2th best pixel estimate = 1th best pixel estimate, for all pixels
  # 0 if absolutly confident for all pixels. One class probability 1, 2th best class probability 0, for all pixels
  """
  BS,C,H,W = pred.shape
  if mask is None:
    mask = torch.ones( (BS,H,W), device=pred.device, dtype=torch.bool)
    
  argm1 = torch.argmax(pred, 1)
  soft1 = torch.nn.functional.softmax(pred, dim=1)

  onehot_argm1 = torch.nn.functional.one_hot(argm1, num_classes=C).permute(0,3,1,2).type(torch.bool)
  ten2 = pred.clone()
  ten2[ onehot_argm1 ] = 0

  argm2 = torch.argmax(ten2, 1)
  onehot_argm2 = torch.nn.functional.one_hot(argm2, num_classes=C).permute(0,3,1,2).type(torch.bool)
  res = [] 

  soft1 = soft1.permute(0,2,3,1)
  onehot_argm1 = onehot_argm1.permute(0,2,3,1)
  onehot_argm2 = onehot_argm2.permute(0,2,3,1)
  
  for b in range(BS):
    res_ = soft1[b][mask[b]][onehot_argm1[b][mask[b]]] - soft1[b][mask[b]][onehot_argm2[b][mask[b]]]
    res.append( res_.mean() )

  return torch.tensor(res, dtype=pred.dtype, device=pred.device)


def test():
  BS,C,H,W = 16,40,300,320
  mask = torch.rand( ( BS,H,W) ) > 0.5
    
  pred = torch.rand( ( BS,C,H,W) )
  res = get_softmax_uncertainty_distance(pred)
  print(res, "should be very low")
  
  pred = torch.rand( ( BS,C,H,W) ) /1000
  pred[:,0,:,:] = 10
  res = get_softmax_uncertainty_distance(pred, mask)
  print(res, 'should be nearly 0')
  
  pred[:,1,:,:] = 8
  res = get_softmax_uncertainty_distance(pred, mask)
  print(res, 'should be betweem 0-1')
test()

In [ ]:
def get_softmax_uncertaintiy_max(pred):
    """
    pred: BS,C,H,W before softmax is applied! 
    
    (1 - max( softmax(pred))) mean over batch size
    
    # 1 if fully uncertain
    # 0 if absolutly confident for all pixels
    """
    BS,C,H,W = pred.shape
    
    argm1 = torch.argmax(pred, 1)
    soft1 = torch.nn.functional.softmax(pred, dim=1)
    onehot_argm1 = torch.nn.functional.one_hot(argm1, num_classes=C).permute(0,3,1,2).type(torch.bool)
    
    soft1 = soft1.permute(0,2,3,1)
    onehot_argm1 = onehot_argm1.permute(0,2,3,1)
    
    res = []
    for b in range(BS):
        res_ = soft1[b][onehot_argm1[b]]
        res.append( torch.mean(res_)  )
#     print(res)
    return torch.tensor(res, dtype=pred.dtype, device=pred.device)

def test():
    BS,C,H,W = 16,40,300,320
    pred = torch.rand( ( BS,C,H,W) )
    res = get_softmax_uncertaintiy_max(pred)
    print(res, "should be very low")
    
    pred = torch.rand( ( BS,C,H,W) ) /1000
    pred[:,0,:,:] = 10
    res = get_softmax_uncertaintiy_max(pred)
    print(res, 'should be nearly 0')
    
    pred[:,1,:,:] = 8
    res = get_softmax_uncertaintiy_max(pred)
    print(res, 'should be betweem 0-1')
    

test()

In [ ]:
from torchvision.utils import make_grid
import os
os.chdir('/home/jonfrey/ASL')
import sys
sys.path.append('/home/jonfrey/ASL/src')
from visu import Visualizer


images = [torch.rand((3,480,640)) for i in range(16)]
labels = [torch.randint(-1,40,(3,480,640)) for i in range(16)]

grid_images = make_grid(images,nrow = 4,padding = 2,
         scale_each = False, pad_value = 0)
grid_labels = make_grid(labels,nrow = 4,padding = 2,
         scale_each = False, pad_value = -1)

print( grid_labels.shape)

visu = Visualizer( p_visu='/home/jonfrey/tmp', logger=None, epoch=0, store=False, num_classes=40)
visu.plot_image(grid_images, jupyter=True)
visu.plot_segmentation( seg = grid_labels[0], jupyter=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
ten = torch.rand( ( 100 ) )

def plot_hist(data, nr_bins=5, x_label='X', y_label='Y', title='Histogramm'):
    if len(data.shape)>1:
        if data.shape[0] == 0:
            data = data[0,:]
        elif data.shape[1] == 0:
            data = data[:,0]
        else:
            raise Exception('plot_hist: Invalid Data Shape')
    if type(data) == torch.Tensor:
        data = list( data.clone().cpu())
    elif type(data) == np.array:
        data = list(data)
    elif type(data) == list:
        pass
    else:
        raise Exception("plot_hist: Unknown Input Type"+str(type(data)))
    
    # the histogram of the data
    n, bins, patches = plt.hist(x = data, bins= nr_bins, density=True, facecolor='g', alpha=0.75)
    print(bins)

    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.grid(True)
    plt.show()

plot_hist(ten)
print(ten)

In [ ]:
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import numpy as np
import matplotlib.pyplot as plt
ten = torch.rand( ( 100 ) )
col = { "red":[255,89,94],
 "yellow":[255,202,58],
 "green":[138,201,38],
 "blue":[25,130,196],
 "purple":[106,76,147] }

li = [ [*(v),255] for v in col.values()]
li = (np.array(li)/255).tolist()
COL_MAP = cm.colors.ListedColormap(li)
COL_MAP(0)

def plot_bar(data, nr_bins=5, x_label='Sample', y_label='Value', title='Bar Plot', sort=True, reverse=True):
    def check_shape(data):
        if len(data.shape)>1:
            if data.shape[0] == 0:
                data = data[0,:]
            elif data.shape[1] == 0:
                data = data[:,0]
            else:
                raise Exception('plot_hist: Invalid Data Shape')
        return data
    
    if type(data) == list:
        pass
    elif type(data) == torch.Tensor:
        data = check_shape(data)
        data = list( data.clone().cpu())
    elif type(data) == np.ndarray:
        data = check_shape(data)
        data = list(data)
    else:
        raise Exception("plot_hist: Unknown Input Type"+str(type(data)))
    
    if sort:
        data.sort(reverse=reverse)
    c = [106,76,147]
    
    col = { "red":[255,89,94],
     "yellow":[255,202,58],
     "green":[138,201,38],
     "blue":[25,130,196],
     "purple":[106,76,147] }

    li = [ [*(v),255] for v in col.values()]
    li = (np.array(li)/255).tolist()
    col_map = cm.colors.ListedColormap(li)

    fig = plt.figure()
    # n, bins, patches = plt.hist(x = data, bins= nr_bins, density=True, facecolor='g', alpha=0.75)
    plt.bar(list(range(len(data))), data, facecolor=COL_MAP(2))
    lab = [ 'ID'+ str(i) for i in range(len(data)) ]

    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.grid(True)
    plt.show()
    fig.show()
    plt.close()
plot_bar(ten.numpy())
# print(ten)
ten
# x = np.arange(4)
# money = [1.5e5, 2.5e6, 5.5e6, 2.0e7]


# def millions(x, pos):
#     'The two args are the value and tick position'
#     return '$%1.1fM' % (x * 1e-6)


# formatter = FuncFormatter(millions)

# fig, ax = plt.subplots()
# ax.yaxis.set_major_formatter(formatter)

# plt.xticks(x, ('Bill', 'Fred', 'Mary', 'Sue'))
# plt.show()

In [ ]:
import cv2
from PIL import Image
# img = np.array(f['dataset'])
# img[img > 1] = 1
i =5
p = f'/media/scratch2/jonfrey/datasets/mlhypersim/ai_026_001/images/scene_cam_00_final_hdf5/frame.000{i}.color.hdf5'
import h5py
with h5py.File(p, 'r') as f:
    img = np.array(f['dataset'])

img = torch.from_numpy(img).type(torch.float32) #.permute( 2, 0, 1)  # C H W
img[img>1] = 1
img = img.numpy()
print(img.max(), img.min() )
H,W,C = img.shape

img = img*255
img = np.uint8(img)
out = cv2.resize( img , dsize=(int(W/2), int(H/2)), interpolation=cv2.INTER_CUBIC)
#out = np.moveaxis(out, [0, 1, 2], [2, 1, 0])
print(out.max(), out.shape, out.dtype)
# out = np.uint8(out)
display(Image.fromarray(out))

np.float32( out )/255 

In [ ]:
indi, counts = torch.unique( torch.randint(0,40,(100,1)) , return_counts = True)


plot_bar(counts.numpy())

In [ ]:
p = "/media/scratch1/jonfrey/models/master_thesis/dev/uncertainty_integration/latent_feature_tensor_0.pt"
data = torch.load( p )

In [ ]:
data.shape


In [ ]:
from torch.nn import functional as F

def get_image_indices(feat, gloable_indices, dis_metric= 'cos',
                     K_aggregate=50, K_return=50, most_dissimilar= True,
                     pick_mode='class_balanced'):
    N, NC, C = feat.shape   
    T = int(N*NC)
    y = torch.range(0,NC-1)[None].repeat(N,1).flatten()
    
    gloable_indices_all = gloable_indices[:,None].repeat(1,NC).flatten()
    feat = feat.view( (T,C) )
    
    # only mark a features as valid if the vector is not 0!
    y[ feat.sum(dim=1) == 0 ] = 999
    
    # create centroids over valid features
    feat_centroids = torch.zeros( (NC,C), device=feat.device, dtype=feat.dtype)
    for i in range(NC):
        feat_centroids[i] = feat[y==i].mean(dim=0)
        
    # create expanded centroid tensor that aligns with feat
    expanded_centroids = feat.clone()
    for i in range(NC):
        m = y == i
        expanded_centroids[ m,: ] = feat_centroids[i][None,:].repeat(T,1)[m, :]
    
    # compute similarity metric
    if dis_metric == 'cos':
        metric = F.cosine_similarity(
            feat.type(torch.float32), 
            expanded_centroids.type(torch.float32),
            dim=1, eps=1e-6) 
    elif dis_metric == 'pairwise':
        metric = F.pairwise_distance(
            feat.type(torch.float32), 
            expanded_centroids.type(torch.float32),
            dim=1, eps=1e-6)
    else:
        raise Exception(f'In get_image_indices dis_metric {dis_metric} not implemented')
    
    if pick_mode== 'most_hits':

        # select K best for each class
        candidates = []
        for i in range(NC):
            m = y==i
            _K = min(m.sum(), K_aggregate)
            values, indices = torch.topk( metric[m] ,_K, 
                largest = not most_dissimilar)
            candidates +=  gloable_indices_all[m][indices].tolist()
        candidates = torch.tensor( candidates )

        # select the globale_image_indicies that are selected most
        indi, counts = torch.unique(candidates, return_counts=True, sorted=False)

        values, indices_of_indices = torch.topk( counts ,K_return) 
        ret_globale_indices = gloable_indices[indices_of_indices]
    
    elif pick_mode == 'class_balanced':
        candidates = []
        nr_features_valid = (y!=999).sum()
        
        for i in range(NC):
            m = y==i
            nr_features_valid_i = (y==i).sum()
            
            factor = float( nr_features_valid_i/nr_features_valid)
            if i == NC-1:
                _K = int(K_return-len(candidates))
            else:
                _K = int( K_return*factor )
                
            if _K > 0:
                max_ele = metric[m].shape[0]
                top_K = min(max_ele, _K+ len(candidates)+1)
                
                values, indices = torch.topk( metric[m] ,_K, 
                    largest = not most_dissimilar)
                added = 0
                for ele in gloable_indices_all[m][indices].tolist():
                    if not ele in candidates and added < _K:
                        added += 1
                        candidates = candidates + [int(ele)]
                
        if len(candidates) > K_return:
            candidates = candidates[:K_return]
        
        # select the globale_image_indicies that are selected most
        ret_globale_indices = torch.tensor(candidates)
        
    return ret_globale_indices
    
ret_gloable_indices = get_image_indices(data, torch.range(0,data.shape[0]-1))
classes = torch.zeros( (data.shape[1]) )
for i in range(ret_gloable_indices.shape[0]):
    idx = int( ret_gloable_indices[i] )
    for n in range( data.shape[1] ):
        if data[idx,n,:].sum() != 0:
            classes[n] += 1
plot_bar(classes.numpy(), sort=False, title='Buffer Label Dist class_balanced cos')

ret_gloable_indices = get_image_indices(data, torch.range(0,data.shape[0]-1), pick_mode='most_hits')
classes = torch.zeros( (data.shape[1]) )
for i in range(ret_gloable_indices.shape[0]):
    idx = int( ret_gloable_indices[i] )
    for n in range( data.shape[1] ):
        if data[idx,n,:].sum() != 0:
            classes[n] += 1
plot_bar(classes.numpy(), sort=False, title='Buffer Label Dist most_hits cos')


classes = torch.zeros( (data.shape[1]) )
for i in range(data.shape[0]):
    idx = int(i)
    for n in range( data.shape[1] ):
        if data[idx,n,:].sum() != 0:
            classes[n] += 1
            
plot_bar(classes.numpy(), sort=False, title='Task Label Dist')

In [ ]:
elements = []
yy = []

image_idx = []

#creates hierachical list of all feature vectors
for ne in range(data.shape[1]):
    classes = []
    labels = []
    for i in range(data.shape[0]):
        if data[i,ne].sum() != 0:
            image_idx.append( i )
            classes.append( data[i,ne] )
            labels.append(ne)
    yy.append(labels)
    print(len(classes))
    elements.append( classes )

len( image_idx )

In [ ]:
# flatten the hierachical list

ele = 0
for e in elements:
    ele += len(e)
X = torch.zeros( (ele,128))
y = torch.zeros( (ele) )

s = 0
for e in elements:
    for i in e:
        X[s] = i
        s+=1 
        
s = 0
for e in yy:
    for i in e:
        y[s] = i
        s+=1 

X = X.numpy()
y = y.numpy()

In [ ]:
X.shape, y.shape

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
pca.fit(X,y)
X_pca = pca.transform(X)

In [ ]:
X.shape, X_pca.shape, X_pca[y==0].shape

X_pca[y==0].shape, y[y==0].shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Fixing random state for reproducibility

N = 100
r0 = 0.6
col = ['r','b','g','y','r','b','g','y']
ma = ['^','^','^','^','o','o','o','o']
nr = 400
for i in range(1):
    if X_pca[y==i,0].shape[0] > nr:
        plt.scatter(X_pca[y==i,0][:nr], X_pca[y==i,1][:nr], marker='^', c=col[i])
    else:
        plt.scatter(X_pca[y==i,0], X_pca[y==i,1], marker='^', c=col[i])
        
display( plt.show() )

# K-Means : Track uncertainty of the samples in the buffer,Measuring overfitting and the perfomance on the buffer

In [ ]:
X_centroids = np.zeros( (data.shape[1],data.shape[2]) )
for i in range(data.shape[1]):
    X_centroids[i] = X[y==i].mean(axis=0)
    
X_centroids [ np.isnan(X_centroids) ] = 0
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

pca = PCA(n_components=3)
pca.fit(X_centroids)
X_centroids_pca = pca.transform(X_centroids)

plt.scatter(X_centroids_pca[:,1], X_centroids_pca[:,2], marker='^', c=col)

display( plt.show() )
X_centroids_pca.shape

In [ ]:
from torch import nn
try:
    y = torch.from_numpy( y ).type(torch.int64)
    X_centroids = torch.from_numpy( X_centroids ).type(torch.float16)
except:
    pass
accum = torch.zeros( (*X.shape), dtype=torch.float16 )

#create the accum tensor with the same size as the full dataset filled with the centroids
for cen in range(X_centroids.shape[0]):
    m = y == cen
    accum[ m,: ] =  X_centroids[cen][None,:].repeat(accum.shape[0],1)[m, :]


cos = nn.CosineSimilarity(dim=1, eps=1e-6)
simil = cos(torch.from_numpy(X).type(torch.float32), accum.type(torch.float32))

print(simil)
K = 50
image_idx = torch.tensor( image_idx )
candidates = []
for cen in range(X_centroids.shape[0]):
    # take the minus for least similar samples
    m = y==cen
    values, indices = torch.topk( -simil[m] ,min(m.sum(), K)) 
    candidates += image_idx[m][indices].tolist()

candidates = torch.tensor( candidates )
indi, counts = torch.unique(candidates, return_counts=True, sorted=False)
K_ = 50
values, indices_of_indices = torch.topk( counts ,K_) 
final_results_of_image_indexes = indi[indices_of_indices]

In [ ]:
m = y == cen 
torch.where(m)[0][0]
accum[torch.where(m)[0][0]] - accum[torch.where(m)[0][100]] 
#LGTM
accum.shape, X.shape

In [ ]:
SCANNET_COLOR_MAP = {
    0: (0.0, 0.0, 0.0),
    1: (174.0, 199.0, 232.0),
    2: (152.0, 223.0, 138.0),
    3: (31.0, 119.0, 180.0),
    4: (255.0, 187.0, 120.0),
    5: (188.0, 189.0, 34.0),
    6: (140.0, 86.0, 75.0),
    7: (255.0, 152.0, 150.0),
    8: (214.0, 39.0, 40.0),
    9: (197.0, 176.0, 213.0),
    10: (148.0, 103.0, 189.0),
    11: (196.0, 156.0, 148.0),
    12: (23.0, 190.0, 207.0),
    14: (247.0, 182.0, 210.0),
    15: (66.0, 188.0, 102.0),
    16: (219.0, 219.0, 141.0),
    17: (140.0, 57.0, 197.0),
    18: (202.0, 185.0, 52.0),
    19: (51.0, 176.0, 203.0),
    20: (200.0, 54.0, 131.0),
    21: (92.0, 193.0, 61.0),
    22: (78.0, 71.0, 183.0),
    23: (172.0, 114.0, 82.0),
    24: (255.0, 127.0, 14.0),
    25: (91.0, 163.0, 138.0),
    26: (153.0, 98.0, 156.0),
    27: (140.0, 153.0, 101.0),
    28: (158.0, 218.0, 229.0),
    29: (100.0, 125.0, 154.0),
    30: (178.0, 127.0, 135.0),
    31: (255.0, 0.0, 135.0),
    32: (146.0, 111.0, 194.0),
    33: (44.0, 160.0, 44.0),
    34: (112.0, 128.0, 144.0),
    35: (96.0, 207.0, 209.0),
    36: (227.0, 119.0, 194.0),
    37: (213.0, 92.0, 176.0),
    38: (94.0, 106.0, 211.0),
    39: (82.0, 84.0, 163.0),
    40: (100.0, 85.0, 144.0),
}
col = np.array( [v for v in SCANNET_COLOR_MAP.values()] )/255
